### 1. 理论知识

1. ID3（Iterative Dichotomiser 3）是一种常用的决策树算法，主要用于**分类**任务。它通过构建树形模型来帮助决策。

2. **特征选择**： ID3 通过计算**信息增益**来选择最佳特征。**信息增益**是指使用某一特征进行分类，所带来的**不确定性减少程度**。ID3 首先计算整个数据集的熵，然后计算在每个特征下的条件熵，信息增益则是二者的差值。

3. **熵**： **熵**是衡量信息不确定性的一个指标。**熵越高，不确定性越大；熵越低，不确定性越小**。ID3 通过计算数据集中每个类别的分布来确定熵。

4. **递归构建**： ID3 采用递归方法来构建树。从根节点开始，选择信息增益最大的特征作为当前节点，然后根据该特征的不同取值分割数据集，继续为每个子集构建子树，直到满足停止条件（如所有样本属于同一类别或没有特征可用）

### 2.数据准备

##### 2.1 导入数据集

In [7]:
# 导入模块
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

# 处理数据集
iris = load_iris()
X = pd.DataFrame(data=iris.data, columns=iris.feature_names)
y = pd.DataFrame(data=iris.target, columns=["label"])


# 特征重命名
map_ = {
    "sepal length (cm)": "花萼长度",
    "sepal width (cm)": "花萼宽度",
    "petal length (cm)": "花瓣长度",
    "petal width (cm)": "花瓣宽度",
}
X = X.rename(columns=map_)
X

,花萼长度,花萼宽度,花瓣长度,花瓣宽度
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


#### 2.2 数据处理

1. ID3树要求特征是**离散**的，因此需要对Iris数据集进行**分箱**操作。

In [8]:
# 对每一列特征进行分箱
X_new = pd.DataFrame()
for c in X.columns:
    binned_data = pd.cut(X[c], bins=4, labels=[0, 1, 2, 3])
    X_new = pd.concat([X_new, binned_data], axis=1)

# Array化，方便计算
X_new = np.array(X_new)
y = np.array(y)
X_new

array([[0, 2, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 2, 0, 0],
       [1, 3, 0, 0],
       [0, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 3, 0, 0],
       [1, 3, 0, 0],
       [1, 3, 0, 0],
       [0, 2, 0, 0],
       [1, 2, 0, 0],
       [0, 2, 0, 0],
       [1, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 1, 0, 0],
       [0, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 2, 0, 0],
       [0, 3, 0, 0],
       [1, 3, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [1, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 1, 0, 0],
       [0, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 2, 0, 0],
       [0, 2, 0, 0],
       [0, 1, 0, 0],
       [0, 2, 0, 0],
       [0, 1,

### 3. 构建ID3树

#### 3.1 熵与信息增益

1. 信息熵的计算：在分类数据集中，信息熵用来衡量 数据集中**标签** 的 **纯度或不确定性**。具体来说，信息熵越高，表示数据集中标签的类别分布越分散，越不纯；信息熵越低，表示标签的类别越集中，数据集越纯。那么，如何计算信息熵，以及信息增益呢？

2. ![Example Image](img/信息熵1.jpg)

3. **将数据集根据某个特征分成若干个子集。假设特征 𝐴 有 𝑉 个可能取值。每个取值将数据集划分成一个子集。**

5. ![Example Image](img/子集信息熵1.jpg)

6. 最后，加权子集信息熵，计算条件熵与信息增益。

7. ![Example Image](img/条件熵与信息增益1.jpg)

1. 需要注意：**子集信息熵公式** = **原数据集信息熵公式**

2. 条件熵 = 子集的信息熵 * 子集比例，如对特征A有4个子集，每个子集的信息熵为[0.2, 0.3, 0.4, 0.1]，每个子集的比例[0.1, 0.5, 0.3, 0.1]

3. 则特征A的条件熵：即以特征A划分时，的子集的熵的加权和 = [0.2*0.1 + 0.3*0.5 + 0.4*0.3 + 0.1*0.1]

In [9]:
# 计算信息熵

# 我写的：
def get_entropy_me(y_):
    entropy = -np.sum(
        [
            (i / y_.shape[0]) * np.log2(i / y_.shape[0])
            for i in np.unique(y_, return_counts=True)[1]
        ]
    )
    return entropy


# gpt写的
def get_entropy_gpt(y):
    _, counts = np.unique(y, return_counts=True)
    probabilities = counts / counts.sum()
    return -np.sum(probabilities * np.log2(probabilities))


# 打印熵值
print(get_entropy_gpt(y))
print(get_entropy_me(y))

1.584962500721156
1.584962500721156


In [10]:
# 计算信息增益

# 我写的
def get_information_gain_me(y_, x_):

    # 根据给定的这一列特征不同值, 找到对应的子集index
    indexs = [np.where(x_ == i) for i in np.unique(x_)] 

    # 根据子集index找到子集y, 求子集y的的信息熵并加权求和
    x_entropy = np.sum([get_entropy_me(y[i]) * (len(y[i]) / len(y)) for i in indexs]) 

    # 返回信息增益
    return get_entropy_me(y_) - x_entropy


# gpt写的
def get_information_gain_gpt(y_, x_):

    # 计算整体熵
    total_entropy = get_entropy_gpt(y_)

    # 计算特征 x 的每个唯一值的熵
    unique_values = np.unique(x_)

    weighted_entropy = 0
    for value in unique_values:

        # 获取特征 x 等于当前值的样本
        subset = y[x_ == value]

        # 计算该子集的熵
        subset_entropy = get_entropy_gpt(subset)

        # 计算该子集的权重
        weight = len(subset) / len(y_)

        # 加权熵
        weighted_entropy += weight * subset_entropy

    # 计算信息增益
    gain = total_entropy - weighted_entropy
    return gain

print(get_information_gain_me(y, X_new[:, 0]))
print(get_information_gain_gpt(y, X_new[:, 0]))

0.6491037267564883
0.6491037267564883


In [11]:
# 构建决策树
def get_best_feature(y_, x_):
    score = []
    for i in range(x_.shape[1]):
        gain_i = get_information_gain_me(y_, x_[:, i])
        score.append(gain_i)

    # 最大信息增益的特征
    return np.argmax(score)


get_best_feature(y, X_new)

np.int64(3)

In [12]:
# sklearn中模型计算
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier(criterion="entropy")
x_train, x_test, y_train, y_test = train_test_split(X_new, y)
model.fit(x_train, y_train)
y_pre = model.predict(x_test)
accuracy_score(y_test, y_pre)

0.9210526315789473